<a href="https://colab.research.google.com/github/claudiocoltellacci/LLM-RDF-for-Finance/blob/main/GraphRAG_%2B_Ontology_Based_Query_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GraphRAG+Ontology-Based Query Check for question answering

This project's final goal is to obtain an assistant for Financial Analysis combining LLMs and RDF Structured Data

## Library import and LLM loading

In [ ]:
! pip install langchain
! pip install langchain_community
! pip install langchain_mistralai
! pip install sentence_transformers
! pip install transformers
! pip install huggingface_hub
! pip install langchain_community
! pip install langchain_mistralai
! pip install langchain_experimental
! pip install mistralai
! pip install getpass
! pip install pypdf
! pip install faiss-gpu
! pip install faiss
! pip install faiss-cpu
! pip install rdflib
! pip install BeautifulSoup
! pip install SPARQLWrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from pathlib import Path
from google.colab import runtime
import pandas as pd
import numpy as np
import torch
from transformers import BitsAndBytesConfig
from huggingface_hub import notebook_login
import re
from sentence_transformers import SentenceTransformer, util
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, SKOS, OWL, RDF, RDFS, VOID, XMLNS, XSD
import langchain
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from rdflib import Graph, RDF, RDFS, Namespace
from rdflib.plugins.sparql import prepareQuery
import requests
from bs4 import BeautifulSoup
import re
import pypdf
import faiss
from mistralai import Mistral
import requests
from getpass import getpass
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from rdflib import Graph, RDF, RDFS, Namespace, Literal
from rdflib.plugins.sparql import prepareQuery
from sentence_transformers import SentenceTransformer
from bs4 import BeautifulSoup
import requests
import re
import os
from typing import Dict, List, Optional
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph, URIRef, RDF, RDFS, Literal
from collections import defaultdict
from typing import Dict, List, Tuple

In [ ]:
from google.colab import userdata
userdata.get('Thesis')
BEE_KEY = userdata.get('Thesis')
client = Mistral(BEE_KEY)

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using the MPS device for MAC")
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA L4


In [ ]:
#Model Verification
def test_mistral_models(api_key):
    models = ["mistral-small-latest", "mistral-medium-latest","open-mistral-nemo","open-codestral-mamba","mistral-large-latest"]  # You can extend this if needed
    url = "https://api.mistral.ai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    for model in models:
        print(f"Testing model: {model}")
        payload = {
            "model": model,
            "messages": [{"role": "user", "content": "Say hello!"}]
        }

        try:
            response = requests.post(url, headers=headers, json=payload)
            if response.status_code == 200:
                print(f"✅ {model} responded successfully.")
                print("Response:", response.json()["choices"][0]["message"]["content"])
            else:
                print(f"❌ {model} failed. Status Code: {response.status_code}")
                print("Error:", response.json())
        except Exception as e:
            print(f"⚠️ Error testing {model}: {e}")

# Example usage:
test_mistral_models(BEE_KEY)

Testing model: mistral-small-latest
✅ mistral-small-latest responded successfully.
Response: Hello! How can I assist you today? Let's have a friendly conversation. 😊 How are you doing?
Testing model: mistral-medium-latest
❌ mistral-medium-latest failed. Status Code: 429
Error: {'message': 'Requests rate limit exceeded'}
Testing model: open-mistral-nemo
❌ open-mistral-nemo failed. Status Code: 429
Error: {'message': 'Requests rate limit exceeded'}
Testing model: open-codestral-mamba
❌ open-codestral-mamba failed. Status Code: 429
Error: {'message': 'Requests rate limit exceeded'}
Testing model: mistral-large-latest
✅ mistral-large-latest responded successfully.
Response: Hello! How are you today? Let's chat about anything you'd like. 😊


## Ontology Loading and Final Cleaning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
Ontology = '/content/drive/MyDrive/ultimate_financial_ontology.ttl'

Mounted at /content/drive


In [ ]:
ontology = Graph()
ontology.parse(Ontology)
print(len(ontology.serialize(format="turtle").split()))

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/rdflib/term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: could not convert string to float: '2,9'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/rdflib/term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: could not convert string to float: '2,5'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/rdflib/term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: could not convert string to float: '0,6'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/rdflib/term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: igno

58087


/usr/local/lib/python3.11/dist-packages/rdflib/term.py:1620: UserWarning: Serializing weird numerical rdflib.term.Literal('0,6', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double'))
  warnings.warn(f"Serializing weird numerical {self!r}")
/usr/local/lib/python3.11/dist-packages/rdflib/term.py:1620: UserWarning: Serializing weird numerical rdflib.term.Literal('2,5', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double'))
  warnings.warn(f"Serializing weird numerical {self!r}")
/usr/local/lib/python3.11/dist-packages/rdflib/term.py:1620: UserWarning: Serializing weird numerical rdflib.term.Literal('2,9', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double'))
  warnings.warn(f"Serializing weird numerical {self!r}")


In [ ]:
Ontology_cleaned = Graph()

for sub, pred, obj in ontology:
  if isinstance (obj, Literal) and obj.datatype == XSD.double:
    try:
      float(obj)
      Ontology_cleaned.add((sub, pred, obj))
    except ValueError:
      fixed_lexical = str(obj).replace(',', '.')
      try:
        new_obj = Literal(float(fixed_lexical), datatype=XSD.double)
        Ontology_cleaned.add((sub, pred, new_obj))
      except ValueError:
        print (f"Still broken: {obj}")
  else:
    Ontology_cleaned.add((sub, pred, obj))

Ontology_cleaned.serialize(destination="Polished_Ontology.ttl", format="turtle")
ontology.parse("Polished_Ontology.ttl")
print(len(Ontology_cleaned.serialize(format="turtle").split()))
full_graph = Ontology_cleaned

58087


## Ontology chunking

Mistral's context window is 128K tokens, which is big but might be not enough for the ontology, also because of the noise produced by the syntaxis, despite being written in Turtle. This is why the Ontology is first cut into several parts based on the Industries.

In [ ]:
# Usa il grafo RDF già pulito
full_graph = Ontology_cleaned

# Cartella di output
output_dir = "industry_modules"
Path(output_dir).mkdir(exist_ok=True)

# Namespace
ns = Namespace("http://art.uniroma2.it/ontologies/financial-ontology#")

# Lista delle industry
industry_list = [
    "Communication_Equipment", "Computer_Hardware", "Consumer_Electronics",
    "Electronic_Components", "Electronics_&_Computer_Distribution", "Information_Technology_Services",
    "Scientific_&_Technical_Instruments", "Semiconductor_Equipment_&_Materials", "Semiconductors",
    "Software_-_Application", "Software_-_Infrastructure", "Solar"
]

# Proprietà rilevanti
IS_IN_INDUSTRY = ns.IsInIndustry
IS_IN_COUNTRY = ns.IsInCountry
IS_IN_REGION = ns.IsInRegion
FINANCIAL_DOC_PROPS = [
    ns.IncomeStatement,
    ns.BalanceSheet,
    ns.CashFlow,
    ns.SharesOutstanding
]

def add_full_description(g: Graph, target_graph: Graph, subject: URIRef):
    """Aggiunge tutte le triple del soggetto, e le label/comment di predicati e oggetti"""
    for p, o in g.predicate_objects(subject=subject):
        target_graph.add((subject, p, o))

        if isinstance(p, URIRef):
            for lp, lv in g.predicate_objects(subject=p):
                if lp in [RDFS.label, RDFS.comment]:
                    target_graph.add((p, lp, lv))

        if isinstance(o, URIRef):
            for pp, oo in g.predicate_objects(subject=o):
                target_graph.add((o, pp, oo))
                if isinstance(pp, URIRef):
                    for lpp, lvv in g.predicate_objects(subject=pp):
                        if lpp in [RDFS.label, RDFS.comment]:
                            target_graph.add((pp, lpp, lvv))

def extract_industry_module(industry_name: str):
    module_graph = Graph()
    industry_uri = ns[industry_name]

    # 🔁 Aggiungi TBox completo (classi + proprietà)
    for s, p, o in full_graph.triples((None, RDF.type, OWL.Class)):
        module_graph.add((s, p, o))
    for s, p, o in full_graph.triples((None, RDF.type, OWL.ObjectProperty)):
        module_graph.add((s, p, o))
    for s, p, o in full_graph.triples((None, RDF.type, OWL.DatatypeProperty)):
        module_graph.add((s, p, o))
    for s, p, o in full_graph.triples((None, RDFS.subClassOf, None)):
        module_graph.add((s, p, o))
    for s, p, o in full_graph.triples((None, RDFS.domain, None)):
        module_graph.add((s, p, o))
    for s, p, o in full_graph.triples((None, RDFS.range, None)):
        module_graph.add((s, p, o))

    # 🔁 Aggiungi etichette e commenti su classi/proprietà
    for t in full_graph.triples((None, RDFS.label, None)):
        module_graph.add(t)
    for t in full_graph.triples((None, RDFS.comment, None)):
        module_graph.add(t)

    # 🔁 Aggiungi le istanze legate all'industry selezionata
    for company in full_graph.subjects(predicate=IS_IN_INDUSTRY, object=industry_uri):
        add_full_description(full_graph, module_graph, company)

        for prop in FINANCIAL_DOC_PROPS:
            for doc in full_graph.objects(company, prop):
                add_full_description(full_graph, module_graph, doc)

        for loc_prop in [IS_IN_COUNTRY, IS_IN_REGION]:
            for loc in full_graph.objects(company, loc_prop):
                add_full_description(full_graph, module_graph, loc)

    output_path = os.path.join(output_dir, f"{industry_name}.ttl")
    module_graph.serialize(destination=output_path, format="turtle")
    print(f"✅ Salvato modulo: {output_path}")

# Avvia generazione dei moduli
for industry in industry_list:
    extract_industry_module(industry)


✅ Salvato modulo: industry_modules/Communication_Equipment.ttl
✅ Salvato modulo: industry_modules/Computer_Hardware.ttl
✅ Salvato modulo: industry_modules/Consumer_Electronics.ttl
✅ Salvato modulo: industry_modules/Electronic_Components.ttl
✅ Salvato modulo: industry_modules/Electronics_&_Computer_Distribution.ttl
✅ Salvato modulo: industry_modules/Information_Technology_Services.ttl
✅ Salvato modulo: industry_modules/Scientific_&_Technical_Instruments.ttl
✅ Salvato modulo: industry_modules/Semiconductor_Equipment_&_Materials.ttl
✅ Salvato modulo: industry_modules/Semiconductors.ttl
✅ Salvato modulo: industry_modules/Software_-_Application.ttl
✅ Salvato modulo: industry_modules/Software_-_Infrastructure.ttl
✅ Salvato modulo: industry_modules/Solar.ttl


In [ ]:
# Esegui estrazione + stampa statistiche per tutte le Industry
for industry in industry_list:
    extract_industry_module(industry)

    # Calcolo statistiche
    file_path = os.path.join(output_dir, f"{industry}.ttl")
    g = Graph()
    g.parse(file_path, format="turtle")

    num_triples = len(g)
    num_companies = len(set(g.subjects(RDF.type, ns.Company)))

    print(f"📊 {industry} → {num_companies} company – {num_triples} triple")

✅ Salvato modulo: industry_modules/Communication_Equipment.ttl
📊 Communication_Equipment → 19 company – 1705 triple
✅ Salvato modulo: industry_modules/Computer_Hardware.ttl
📊 Computer_Hardware → 18 company – 1679 triple
✅ Salvato modulo: industry_modules/Consumer_Electronics.ttl
📊 Consumer_Electronics → 20 company – 1844 triple
✅ Salvato modulo: industry_modules/Electronic_Components.ttl
📊 Electronic_Components → 19 company – 1767 triple
✅ Salvato modulo: industry_modules/Electronics_&_Computer_Distribution.ttl
📊 Electronics_&_Computer_Distribution → 8 company – 943 triple
✅ Salvato modulo: industry_modules/Information_Technology_Services.ttl
📊 Information_Technology_Services → 20 company – 1817 triple
✅ Salvato modulo: industry_modules/Scientific_&_Technical_Instruments.ttl
📊 Scientific_&_Technical_Instruments → 19 company – 1704 triple
✅ Salvato modulo: industry_modules/Semiconductor_Equipment_&_Materials.ttl
📊 Semiconductor_Equipment_&_Materials → 20 company – 1786 triple
✅ Salvato 

In [ ]:
from google.colab import files
import shutil

shutil.make_archive("industry_modules", 'zip', "industry_modules")
files.download("industry_modules.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Ontology Textualization

LLMs are trained on Texts. Therefore, to get better results, it might be useful to convert the sub-Knowledge Graphs into texts

In [ ]:
# Impostazioni
module_dir = "industry_modules"
output_dir = "industry_texts"
Path(output_dir).mkdir(exist_ok=True)

# Setup LLM
llm = ChatMistralAI(model="open-mistral-nemo", api_key=BEE_KEY)

# Prompt template
prompt_template = PromptTemplate(
    input_variables=["ontology"],
    template=(
"""You are a financial analyst and ontology expert. Given the following ontology module in Turtle format, generate a **detailed and comprehensive textual description** of each company. Your goal is to **translate every RDF triple**
    related to each company into **clear, natural language**.

For **each company**, follow this structure:

1. **General Overview**: Include revenues and costs variation, market capitalisation, future investments, amortisations, and dividends distributed to shareholders.
2. **Context**:
   - Describe the **industry** they belong to and the **country** they operate in.
   - Include any available indexes, labels, or other metadata.
3. **Financial Statements**:
   - **Income Statement**: Describe all available line items (e.g., EBIT, EBITDA, Normalized EBITDA, Tax Rate, Total Unusual Items, Net Income, Normalized Income, Interest Expense, Reconciled Cost of Revenue, Reconciled Depreciation,
       Tax Effect of Unusual Items, Total Unusual Items, Total Unusual Items Excluding Goodwill).
   - **Balance Sheet**: Describe all available line items (e.g. Cash and Cash Equivalents, Other Short Term Investments, Accounts Receivable, Allowance for Doubtful Accounts, Net Debt, Total Debt, Share Issued, Ordinary Shares Number,
       Treasury Shares Number, Tangible Book Value).
   - **Cash Flow Statement**: Describe all available line items (e.g. Beginning Cash Position, End Cash Position, Free Cash Floe, Supplemental Data, Issuance of Debt, Repayment of Debt, Issuance of Capital Stock, Capital Expenditure,
        Changes in Cash, Income Tax Paid Supplemental Data, Interest Paid Supplemental Data).
4. **Shares Outstanding**: Describe the number and variation if available.

**IMPORTANT**:
- **Rephrase** every RDF statement involving the company, its documents, and their properties in human-readable language.
- Do **not** skip any RDF predicate or literal.
- Do **not** use RDF or ontology-specific terms (e.g., "object property", "owl:Class"). Focus on financial storytelling.

Ontology Module:
{ontology}"""
    )
)

chain = LLMChain(llm=llm, prompt=prompt_template)

# Loop sui moduli
#for ttl_file in Path(module_dir).glob("*.ttl"):
 #   print(f"🔍 Verbalizing: {ttl_file.name}")
  #  g = Graph()
   # g.parse(str(ttl_file), format="turtle")

    #turtle_content = g.serialize(format="turtle")
    #try:
     #   explanation = chain.run(ontology=turtle_content).strip()
    #except Exception as e:
     #   explanation = f"⚠️ Error generating text: {e}"

    #out_path = Path(output_dir) / (ttl_file.stem + ".txt")
    #with open(out_path, "w", encoding="utf-8") as f:
     #   f.write(explanation)

    #print(f"✅ Saved: {out_path}")


<ipython-input-12-c23a8b0609a5>:41: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


In [ ]:
sa= "/content/industry_modules/Software_-_Application.ttl"
g = Graph()
g.parse(sa, format="turtle")
turtle_content = g.serialize(format="turtle")
explanation = chain.run(ontology=turtle_content).strip()
print(explanation)
out_path = Path(output_dir) / (Path(sa).stem + ".txt")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(explanation)
print(f"✅ Saved: {out_path}")


Based on the provided RDF triples, here's a structured and comprehensive textual description of each company, following the requested format:

1. **General Overview:**
   - **Revenue Variation:** 0.063 (0.63%) for most companies, indicating a steady growth in revenue.
   - **Market Capitalization:** Varies from 1.13 billion to 3.42 billion, with an average of 2.14 billion.
   - **Future Investments:** Ranges from 0 to 2.09 billion, with an average of 0.86 billion.
   - **Amortization:** Ranges from 0 to 4.196 million, with an average of 1.04 million.

2. **Context:**
   - **Industry:** Most companies belong to the 'Software - Application' industry, which is a sub-sector of 'Technology'.
   - **Country:** Companies are headquartered in the United States, which is part of the North America region/continent.
   - **Market Return:** The average market return for the 'Technology' sector is 2.363 (23.63%).

3. **Financial Statements:**
   - **Income Statement:**
     - **EBIT:** Ranges from 

In [ ]:
si= "/content/industry_modules/Software_-_Infrastructure.ttl"
g = Graph()
g.parse(si, format="turtle")
turtle_content = g.serialize(format="turtle")
explanation = chain.run(ontology=turtle_content).strip()
print(explanation)
out_path = Path(output_dir) / (Path(si).stem + ".txt")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(explanation)
print(f"✅ Saved: {out_path}")

<ipython-input-13-0f3bf2aa4b6e>:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  explanation = chain.run(ontology=turtle_content).strip()


**Detailed and Comprehensive Textual Descriptions:**

1. **Adobe Inc.**
   - **Balance Sheet:**
     - Cash and Cash Equivalents: $7.141 billion (30/11/2023), $8.166895 billion (31/12/2023)
     - Ordinary Shares Number: 4.55 billion (30/11/2023), 4.46685 billion (31/12/2023)
     - Total Debt: $4.08 billion (30/11/2023), $6.07157 billion (31/12/2023)
     - Tangible Book Value: $9.151 billion (30/11/2023), $9.15147 billion (31/12/2023)
   - **Cash Flow:**
     - Free Cash Flow: $6.942 billion (30/11/2023)
     - Issuance of Debt: $0 (30/11/2023)
     - Repayment of Debt: -$5.019 billion (30/11/2023)
   - **Income Statement:**
     - EBIT: $6.912 billion (30/11/2023)
     - EBITDA: $7.784 billion (30/11/2023)
     - Net Income: $5.428 billion (30/11/2023)
     - Tax Rate: 20.1%
     - Total Unusual Items: -$8.006 million (30/11/2023)
   - **Shares Outstanding:** 4.717 billion (25/11/2022), 5.10038 billion (13/12/2023)

2. **Affirm Holdings, Inc.**
   - **Balance Sheet:**
     - Cash an

In [ ]:
so= "/content/industry_modules/Solar.ttl"
g = Graph()
g.parse(so, format="turtle")
turtle_content = g.serialize(format="turtle")
explanation = chain.run(ontology=turtle_content).strip()
print(explanation)
out_path = Path(output_dir) / (Path(so).stem + ".txt")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(explanation)
print(f"✅ Saved: {out_path}")

**Array Technologies, Inc.**

**General Overview:**
- Revenue Variation: 0.03
- Market Capitalization: 8.660728e+08
- Future Investments: 0.00
- Amortizations: 0.00
- Dividends Distributed: 0.00

**Context:**
- Industry: Solar
- Country: United States
- Continent: North America
- GDP of Country: 2.54397e+07
- Equity Risk Premium of Country: 5.333417e-02
- Inflation Rate of Country: 2.9
- Moody's Rating of Country: Aa3
- PRS Score of Country: 7.025
- S&P Rating of Country: AA+
- Tax Rate of Country: 2.5e-01
- Fertility Rate of Country: 1.6
- GDP invested in research of Country: 0.0346
- Sum of Market Capitalizations of Companies in Country: 4.915704e+07
- World Digital Competitiveness Ranking (WDCR) of Country: 9.131
- Yearly Increase/Decrease Rate of Country's population: 5.7e-01

**Financial Statements:**

*Income Statement:*
- EBIT: 2.21386e+08
- EBITDA: 2.76212e+08
- Interest Expense: 4.4229e+07
- Net Income from Continuing Operation Net Minority Interest: 1.3724e+08
- Net Interest 

In [ ]:
sc= "/content/industry_modules/Semiconductors.ttl"
g = Graph()
g.parse(sc, format="turtle")
turtle_content = g.serialize(format="turtle")
explanation = chain.run(ontology=turtle_content).strip()
print(explanation)
out_path = Path(output_dir) / (Path(sc).stem + ".txt")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(explanation)
print(f"✅ Saved: {out_path}")

**General Overview:**

1. **Revenue Variation:** Most companies experienced positive revenue variation, with NVIDIA Corporation (3.41139e+12) and Taiwan Semiconductor Manufacturing Company Limited (9.91316e+11) showing the highest increases. Micron Technology, Inc. (1.15731e+09) had the lowest revenue variation.

2. **Market Capitalization:** NVIDIA Corporation (3.41170e+12) and Taiwan Semiconductor Manufacturing Company Limited (9.93017e+11) had the highest market capitalizations. Micron Technology, Inc. (1.15731e+09) had the lowest.

3. **Cash Flow:** NVIDIA Corporation (2.7021e+10) and Taiwan Semiconductor Manufacturing Company Limited (2.86569e+11) had the highest free cash flows. Micron Technology, Inc. (1.21e+08) had the lowest.

4. **Shares Outstanding:** NVIDIA Corporation (2.48413e+10) and Taiwan Semiconductor Manufacturing Company Limited (5.18655e+09) had the highest number of shares outstanding. Micron Technology, Inc. (1.15731e+09) had the lowest.

5. **Context:** The comp

In [ ]:
se= "/content/industry_modules/Semiconductor_Equipment_&_Materials.ttl"
g = Graph()
g.parse(se, format="turtle")
turtle_content = g.serialize(format="turtle")
explanation = chain.run(ontology=turtle_content).strip()
print(explanation)
out_path = Path(output_dir) / (Path(se).stem + ".txt")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(explanation)
print(f"✅ Saved: {out_path}")

Based on the provided RDF triples, here's a detailed and comprehensive textual description of each company, including their financial statements, context, and shares outstanding:

1. **ASML Holding N.V.**
   - **Balance Sheet (31/12/2021, 31/12/2023)**
     - Cash and Cash Equivalents: 7.0047e+09 (2023), 7.1129e+09 (2021)
     - Ordinary Shares Number: 3.934217e+08 (2023), 3.99529e+08 (2021)
     - Total Debt: 4.646127e+09 (2023), 5.8145e+07 (2021)
   - **Cash Flow (31/12/2023)**
     - Changes in Cash: -2.498e+08
     - End Cash Position: 7.0047e+09
     - Free Cash Flow: 3.2472e+07
   - **Income Statement (31/12/2023)**
     - EBIT: 9.2362e+09
     - EBITDA: 9.976e+09
     - Net Income: 7.839e+09
     - Tax Rate: 1.6e-01
   - **Shares Outstanding (31/12/2021, 31/12/2023)**
     - Number and Variation: 3.934217e+08 (2023), 3.99529e+08 (2021)

2. **Ambarella, Inc.**
   - **Balance Sheet (31/01/2022, 31/01/2024)**
     - Gross Accounts Receivable: 1.779e+09 (2024), 8.9657e+08 (2022)
   

In [ ]:
st= "/content/industry_modules/Scientific_&_Technical_Instruments.ttl"
g = Graph()
g.parse(st, format="turtle")
turtle_content = g.serialize(format="turtle")
explanation = chain.run(ontology=turtle_content).strip()
print(explanation)
out_path = Path(output_dir) / (Path(st).stem + ".txt")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(explanation)
print(f"✅ Saved: {out_path}")

Based on the provided RDF triples, here's a detailed and comprehensive textual description of each company, following the requested structure:

**1. General Overview:**

- **Acorn Energy, Inc.**
  - Revenue Variation: 3.4%
  - Market Capitalization: $4.4 billion
  - Future Investments: Not provided
  - Amortizations: Not provided
  - Dividends: Not provided

- **Badger Meter, Inc.**
  - Revenue Variation: 3.2%
  - Market Capitalization: $5.1 billion
  - Future Investments: Not provided
  - Amortizations: Not provided
  - Dividends: $0.02 per share

- **Cognex Corporation**
  - Revenue Variation: 3.2%
  - Market Capitalization: $6.5 billion
  - Future Investments: Not provided
  - Amortizations: Not provided
  - Dividends: $0.08 per share

- **Coherent Corp.**
  - Revenue Variation: 3.2%
  - Market Capitalization: $1.7 billion
  - Future Investments: Not provided
  - Amortizations: Not provided
  - Dividends: $0.12 per share

- **ESCO Technologies Inc.**
  - Revenue Variation: 3.2%
  - 

In [ ]:
it= "/content/industry_modules/Information_Technology_Services.ttl"
g = Graph()
g.parse(it, format="turtle")
turtle_content = g.serialize(format="turtle")
explanation = chain.run(ontology=turtle_content).strip()
print(explanation)
out_path = Path(output_dir) / (Path(it).stem + ".txt")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(explanation)
print(f"✅ Saved: {out_path}")

Based on the provided RDF triples, here's a detailed and comprehensive textual description of each company, including their financial statements, context, and shares outstanding:

**Accenture plc**
- **General Overview:** Accenture plc is a multinational professional services company headquartered in Ireland, operating in the Information Technology Services sector. It has a market capitalization of 2.14758e+11 as of 2022-12-31.
- **Context:**
  - **Industry:** Information Technology Services
  - **Country:** Ireland
  - **Region/Continent:** Europe
- **Financial Statements:**
  - **Income Statement (2022-12-31):** Revenue of 7.264787e+09, Net Income of 7.264787e+09, Normalized Income of 7.264787e+09, EBIT of 9.758292e+09, EBITDA of 1.118833e+10, Interest Expense of 5.8969e+07, Tax Rate of 2.4%, Total Unusual Items of 0e+00, and Total Unusual Items Excluding Goodwill of 0e+00.
  - **Balance Sheet (2022-12-31):** Cash and Cash Equivalents of 5.004469e+09, Other Short Term Investments of 

In [ ]:
et= "/content/industry_modules/Electronics_&_Computer_Distribution.ttl"
g = Graph()
g.parse(et, format="turtle")
turtle_content = g.serialize(format="turtle")
explanation = chain.run(ontology=turtle_content).strip()
print(explanation)
out_path = Path(output_dir) / (Path(et).stem + ".txt")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(explanation)
print(f"✅ Saved: {out_path}")

**Arrow Electronics, Inc.**

**General Overview:**
- Revenue Variation: 0.0145
- Market Capitalization: 6.472294e+09
- Future Investments: 0.0275
- Amortizations: 0.0275
- Dividends: 0e+00

**Context:**
- Industry: Electronics & Computer Distribution
- Country: United States
- Indexes, Labels, or Other Metadata: None available

**Financial Statements:**

*Income Statement:*
- EBIT: 1.493078e+09
- EBITDA: 1.674194e+09
- Interest Expense: 3.28724e+08
- Net Income: 9.03505e+08
- Normalized EBITDA: 1.738826e+09
- Normalized Income: 9.539826e+08
- Reconciled Cost of Revenue: 2.89581e+10
- Reconciled Depreciation: 1.81116e+08
- Tax Effect of Unusual Items: -1.415441e+07
- Tax Rate for Calculations: 0.22
- Total Unusual Items: -6.4632e+07
- Total Unusual Items Excluding Goodwill: -6.4632e+07

*Balance Sheet:*
- Cash and Cash Equivalents: 2.18053e+08
- Cash, Cash Equivalents, and Other Short Term Investments: 2.18053e+08
- Gross Accounts Receivable: 3.4804e+09
- Net Debt: 4.2199e+10
- Ordinary

In [ ]:
ec= "/content/industry_modules/Electronic_Components.ttl"
g = Graph()
g.parse(ec, format="turtle")
turtle_content = g.serialize(format="turtle")
explanation = chain.run(ontology=turtle_content).strip()
print(explanation)
out_path = Path(output_dir) / (Path(ec).stem + ".txt")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(explanation)
print(f"✅ Saved: {out_path}")

Based on the provided RDF triples, here are the detailed and comprehensive textual descriptions for each company:

1. **Amphenol Corporation**
   - **General Overview**: Amphenol Corporation is a company operating in the electronic components industry, with headquarters in the United States. It has a market capitalization of $8.833289e+09 and a revenue variation of 0.072. Its cost variation is 0.0275, and it has a dividend yield of 0.000.
   - **Context**:
     - **Industry**: Electronic Components
     - **Country**: United States
     - **Sector**: Technology
     - **Region**: North America
   - **Financial Statements**:
     - **Income Statement**: Amphenol Corporation's EBIT for the year ending December 31, 2023, was $2.5943e+09, with an EBITDA of $3.0007e+09. Its net income was $1.928e+09, with a tax rate of 2.1%. Its total unusual items were -$2.92e+07, leading to a net income of $1.951156e+08 after unusual items.
     - **Balance Sheet**: As of December 31, 2023, Amphenol Corpo

In [ ]:
ce= "/content/industry_modules/Consumer_Electronics.ttl"
g = Graph()
g.parse(ce, format="turtle")
turtle_content = g.serialize(format="turtle")
explanation = chain.run(ontology=turtle_content).strip()
print(explanation)
out_path = Path(output_dir) / (Path(ce).stem + ".txt")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(explanation)
print(f"✅ Saved: {out_path}")

Based on the provided RDF triples, here are the detailed and comprehensive textual descriptions for each company, following the given structure:

1. **General Overview**
   - **Revenue Variation**: The revenue variation is not provided in the given RDF triples.
   - **Context**:
     - **Industry**: These companies belong to the Consumer Electronics industry.
     - **Country**: The companies are headquartered in various countries, including the United States, Japan, China, India, and more.
     - **Region/Continent**: The companies are located in different regions and continents, such as North America, Asia, and Europe.
   - **Financial Statements**:
     - **Income Statement**: The income statements provide information about the companies' earnings, expenses, and taxes for the given periods.
     - **Balance Sheet**: The balance sheets show the assets, liabilities, and equity of the companies as of the given dates.
     - **Cash Flow Statement**: The cash flow statements detail the i

In [ ]:
ch= "/content/industry_modules/Computer_Hardware.ttl"
g = Graph()
g.parse(ch, format="turtle")
turtle_content = g.serialize(format="turtle")
explanation = chain.run(ontology=turtle_content).strip()
print(explanation)
out_path = Path(output_dir) / (Path(ch).stem + ".txt")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(explanation)
print(f"✅ Saved: {out_path}")

Based on the provided RDF triples, here's a detailed and comprehensive textual description of each company, following the structure you provided:

1. **General Overview**
   - **Revenue and Cost Variation**: ASUSTeK Computer's revenue and cost variation are not provided. Dell Technologies' revenue variation is -0.14, indicating a decrease, while its cost variation is -0.07, showing a slight increase in costs.
   - **Market Capitalization**: ASUSTeK Computer's market capitalization is 1.370111e+10, while Dell Technologies' market capitalization is 8.313747e+10.
   - **Future Investments**: ASUSTeK Computer's future amortization is 0.0275, indicating minimal future investments. Dell Technologies' future investments are not provided.

2. **Context**
   - **Industry**: Both ASUSTeK Computer and Dell Technologies belong to the Computer Hardware industry.
   - **Country**: ASUSTeK Computer is headquartered in Taiwan, while Dell Technologies is based in the United States.
   - **Region/Contin

In [ ]:
com= "/content/industry_modules/Communication_Equipment.ttl"
g = Graph()
g.parse(com, format="turtle")
turtle_content = g.serialize(format="turtle")
explanation = chain.run(ontology=turtle_content).strip()
print(explanation)
out_path = Path(output_dir) / (Path(com).stem + ".txt")
with open(out_path, "w", encoding="utf-8") as f:
    f.write(explanation)
print(f"✅ Saved: {out_path}")

Based on the provided RDF triples, here's a detailed and comprehensive textual description of each company, following the structure you provided:

**1. General Overview**

- **AST SpaceMobile, Inc.**
  - Revenue Variation: 0.06
  - Market Capitalization: 6.826148e+09
  - Future Investments: 1000000000
  - Amortizations: 2000000000
  - Cost Variation: 0.0275
  - Dividends: 0e+00
  - Shares Outstanding: 1.8153e+08 to 2.92591e+08
  - Headquarters: United States
  - Industry: Communication Equipment
  - Sector: Technology

- **Airgain, Inc.**
  - Revenue Variation: 0.06
  - Market Capitalization: 9.597582e+07
  - Revenue: 2.442e+09
  - Costs: 2.442e+09
  - Profit: 0e+00
  - Dividends: 0e+00
  - Shares Outstanding: 1.0097e+07 to 1.15773e+07
  - Headquarters: United States
  - Industry: Communication Equipment
  - Sector: Technology

- **Applied Optoelectronics, Inc.**
  - Revenue Variation: 0.06
  - Market Capitalization: 1.518014e+09
  - Revenue: 2.442e+09
  - Costs: 2.442e+09
  - Profit: 

#Prompt Similarity

The assistant should be able to recognise which sub-Knowledge Graph is fit for the information retrieval task. We will use spaCy's functions

In [ ]:
question = "What's the financial situation of Oracle?"

Light model but could be ineffective

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 133.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")  # Modello leggero ma efficace, devi scrivere il nome esatto della società

def extract_company_name(question: str) -> str:
    doc = nlp(question)
    for ent in doc.ents:
        if ent.label_ == "ORG":
            return ent.text
    return question.strip()

Heavy model but most certainly effective

In [ ]:
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 54.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

nlp = spacy.load("en_core_web_trf")  # Modello pesante ma estremamente efficace

def extract_company_name(question: str) -> str:
    doc = nlp(question)
    for ent in doc.ents:
        if ent.label_ == "ORG":
            return ent.text
    return question.strip()

In [ ]:
print(extract_company_name(question))
company_name = extract_company_name(question)
print(company_name)

Oracle
Oracle


In [ ]:
for s,p,o in full_graph.triples((None, RDFS.label, None)):
  print(s,p,o)

http://art.uniroma2.it/ontologies/financial-ontology#BSGrossAccountsReceivable http://www.w3.org/2000/01/rdf-schema#label Gross Accounts Receivable - Balance Sheet line item
http://art.uniroma2.it/ontologies/financial-ontology#FutureAmortization http://www.w3.org/2000/01/rdf-schema#label Amortisation to be paid due to Future Investments
http://art.uniroma2.it/ontologies/financial-ontology#macroIndustry http://www.w3.org/2000/01/rdf-schema#label Industry belonging to a Sector
http://art.uniroma2.it/ontologies/financial-ontology#FutureInvestments http://www.w3.org/2000/01/rdf-schema#label Future Investments programmed by the company
http://art.uniroma2.it/ontologies/financial-ontology#ISNetInterestIncome http://www.w3.org/2000/01/rdf-schema#label Net Interest Income - Income Statement line item
http://art.uniroma2.it/ontologies/financial-ontology#CountryGDP http://www.w3.org/2000/01/rdf-schema#label Gross Domestic Product (GDP) of a Country
http://art.uniroma2.it/ontologies/financial-ont

In [ ]:
def extract_company_labels_custom_label(graph: Graph) -> dict:
    """
    Extracts rdf:label values from instances of ns1:Company (financial-ontology#Company),
    where labels are stored under rdf:label instead of rdfs:label.
    """


    ns1 = Namespace("http://art.uniroma2.it/ontologies/financial-ontology#")
    rdf_label = URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#label")  # your dataset's label usage

    companies = {}

    for s in graph.subjects(RDF.type, ns1.Company):
        labels = [str(o) for o in graph.objects(s, rdf_label) if isinstance(o, Literal)]
        if labels:
            companies[s] = labels

    return companies

In [ ]:
labels_dict = extract_company_labels_custom_label(full_graph)

print(labels_dict)

{rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#WidePoint_Corporation'): ['WidePoint Corporation', 'WidePoint'], rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Cisco_Systems,_Inc.'): ['Cisco Systems', 'Cisco Systems, Inc.'], rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Novanta_Inc.'): ['Novanta', 'Novanta Inc.'], rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Priority_Technology_Holdings,_Inc.'): ['Priority Technology Holdings, Inc.', 'Priority Technology s'], rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Turtle_Beach_Corporation'): ['Turtle Beach', 'Turtle Beach Corporation'], rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Kyndryl_Holdings,_Inc.'): ['Kyndryl Holdings, Inc.', 'Kyndryl s'], rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Juniper_Networks,_Inc.'): ['Juniper Networks', 'Juniper Networks, Inc

In [ ]:
companies = extract_company_labels_custom_label(full_graph)
for uri, labels in companies.items():
    print(f"🏢 {uri}")
    for label in labels:
        print(f"   🔹 {label}")

🏢 http://art.uniroma2.it/ontologies/financial-ontology#WidePoint_Corporation
   🔹 WidePoint Corporation
   🔹 WidePoint
🏢 http://art.uniroma2.it/ontologies/financial-ontology#Cisco_Systems,_Inc.
   🔹 Cisco Systems
   🔹 Cisco Systems, Inc.
🏢 http://art.uniroma2.it/ontologies/financial-ontology#Novanta_Inc.
   🔹 Novanta
   🔹 Novanta Inc.
🏢 http://art.uniroma2.it/ontologies/financial-ontology#Priority_Technology_Holdings,_Inc.
   🔹 Priority Technology Holdings, Inc.
   🔹 Priority Technology s
🏢 http://art.uniroma2.it/ontologies/financial-ontology#Turtle_Beach_Corporation
   🔹 Turtle Beach
   🔹 Turtle Beach Corporation
🏢 http://art.uniroma2.it/ontologies/financial-ontology#Kyndryl_Holdings,_Inc.
   🔹 Kyndryl Holdings, Inc.
   🔹 Kyndryl s
🏢 http://art.uniroma2.it/ontologies/financial-ontology#Juniper_Networks,_Inc.
   🔹 Juniper Networks
   🔹 Juniper Networks, Inc.
🏢 http://art.uniroma2.it/ontologies/financial-ontology#Analog_Devices,_Inc.
   🔹 Analog Devices, Inc.
   🔹 Analog Devices
🏢 http:

In [ ]:
def find_company_module(question: str, ontology_dir: str = "industry_modules") -> str:
    ns1 = Namespace("http://art.uniroma2.it/ontologies/financial-ontology#")
    rdf_label = URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#label")
    company_name = extract_company_name(question)

    for ttl_file in Path(ontology_dir).glob("*.ttl"):
        graph = Graph()
        graph.parse(str(ttl_file), format="turtle")

        for s in graph.subjects(RDF.type, ns1.Company):
          labels = [str(o) for o in graph.objects(s, rdf_label) if isinstance(o, Literal)]
          if any(company_name.lower() in label.lower() for label in labels):
            return ttl_file.name

    return None

module_found = find_company_module(question)
if module_found:
    print(f"✅ Company trovata nel modulo: {module_found}")
else:
    print("❌ Nessuna corrispondenza trovata")

✅ Company trovata nel modulo: Software_-_Infrastructure.ttl


In [ ]:
from pathlib import Path

def get_matching_paths(module_name: str,
                       ttl_dir="industry_modules",
                       txt_dir="industry_texts") -> tuple:
    ttl_path = Path(ttl_dir) / f"{module_name}.ttl"
    txt_path = Path(txt_dir) / f"{module_name}.txt"
    return str(ttl_path), str(txt_path)

In [ ]:
ttl_file, txt_file = get_matching_paths(Path(module_found).stem)

print(f"TTL file: {ttl_file}")
print(f"TXT file: {txt_file}")

TTL file: industry_modules/Software_-_Infrastructure.ttl
TXT file: industry_texts/Software_-_Infrastructure.txt


In [ ]:
# Carica il grafo per SPARQL
g = Graph().parse(ttl_file, format="turtle")

In [ ]:
print(g.serialize(format="turtle"))

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix ns1: <http://art.uniroma2.it/ontologies/financial-ontology#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://art.uniroma2.it/ontologies/financial-ontology#Adobe_Inc.> a ns1:Company ;
    ns1:BalanceSheet <http://art.uniroma2.it/ontologies/financial-ontology#balSh_Adobe_Inc._30/11/2021>,
        <http://art.uniroma2.it/ontologies/financial-ontology#balSh_Adobe_Inc._30/11/2023> ;
    ns1:CashFlow <http://art.uniroma2.it/ontologies/financial-ontology#cashFlow_Adobe_Inc._30/11/2023> ;
    ns1:CostVariation "0.192"^^owl:real ;
    ns1:Dividends 0e+00 ;
    ns1:IncomeStatement <http://art.uniroma2.it/ontologies/financial-ontology#incStat_Adobe_Inc._30/11/2023> ;
    ns1:IsInCountry ns1:United_States ;
    ns1:IsInIndustry ns1:Software_-_Infrastructure ;
    ns1:

#Ontology-Based Query Check

Now the Ontology is used for writing a SPARQL query which will be written, checked and edited by the LLM

In [ ]:
def generate_sparql_with_mistral(companyname: str, ontology_ttl: str) -> str:
    llm = ChatMistralAI(model="codestral-latest", api_key=BEE_KEY)
    prompt_template = PromptTemplate(
        input_variables=["ontology", "company_name"],
        template=("""Given the OWL model described in the following TTL file:
                      {ontology}

                      And the name of a company:
                      {company_name}

                      Replace 'placeholder' in the SPARQL query below to filter results for the given company name. Ensure that the syntax is valid and the query is executable.

                      Return ONLY the updated SPARQL query—do not explain anything.

                      PREFIX financial: <http://art.uniroma2.it/ontologies/financial-ontology#>
                      PREFIX dc: <http://purl.org/dc/elements/1.1/>
                      PREFIX grddl: <http://www.w3.org/2003/g/data-view#>
                      PREFIX owl: <http://www.w3.org/2002/07/owl#>
                      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

                      SELECT * WHERE {{
                        VALUES ?statementType {{
                          financial:MarketCapitalization
                          financial:Dividends
                          financial:RevenueVariation
                          financial:CostVariation
                          financial:IsInCountry
                          financial:IsInIndustry
                          financial:IsInSector
                          financial:IncomeStatement
                          financial:BalanceSheet
                          financial:CashFlow
                          financial:SharesOutstanding
                        }}
                        ?company rdf:label ?label.
                        FILTER(regex(str(?label), "placeholder", "i"))
                        ?company ?statementType ?statement .
                        ?statement ?property ?value .
                        FILTER (isLiteral(?value))
                      }}
                      ORDER BY ?statementType ?statement ?property
                      """
            )
    )

    chain = LLMChain(llm=llm, prompt=prompt_template)
    result = chain.run({"ontology": ontology_ttl, "company_name": company_name})
    print(result)  # This will print the generated SPARQL query
    return result.strip()  # Return the query, stripping any extra whitespace

def check_domain_violation(query_graph: Graph, ontology_graph: Graph) -> list:
    violations = []
    for s, p, o in query_graph.triples((None, None, None)):
        domain = ontology_graph.value(subject=p, predicate=RDFS.domain)
        if domain and not (s, RDF.type, domain) in query_graph:
            violations.append({
                "type": "domain",
                "property": p,
                "expected_domain": domain,
                "subject": s
            })
    return violations

def check_range_violation(query_graph: Graph, ontology_graph: Graph) -> list:
    violations = []
    for s, p, o in query_graph.triples((None, None, None)):
        range_ = ontology_graph.value(subject=p, predicate=RDFS.range)
        if range_ and not (o, RDF.type, range_) in query_graph:
            violations.append({
                "type": "range",
                "property": p,
                "expected_range": range_,
                "object": o
            })
    return violations

def explain_violation(violation: dict) -> str:
    if violation["type"] == "domain":
        return (f"The property {violation['property']} has domain {violation['expected_domain']}, "
                f"but its subject {violation['subject']} does not match or inherit from it.")
    elif violation["type"] == "range":
        return (f"The property {violation['property']} has range {violation['expected_range']}, "
                f"but its object {violation['object']} does not match or inherit from it.")
    return "Unknown violation."

def repair_query_with_mistral(bad_query: str, ontology_str: str, violations: list, api_key: str) -> str:
    llm = ChatMistralAI(model="codestral-latest", api_key=api_key)
    prompt = PromptTemplate(
        input_variables=["ontology", "query", "issues"],
        template=("You are an expert in RDF and SPARQL. Given the following ontology in Turtle format:\n\n"
                  "{ontology}\n\n"
                  "The following SPARQL query has the following issues:\n\n"
                  "{issues}\n\n"
                  "Please rewrite the query to fix the identified issues. ONLY return the corrected SPARQL query.\n"
                  "If no issues are found or the query is already correct, return the original query unchanged.\n\n"
                  "Query:\n{query}")
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run({
        "ontology": ontology_str,
        "query": bad_query,
        "issues": "\n".join(violations)
    })
    return response.strip()


In [ ]:
import re

def clean_sparql_output(raw_query: str) -> str:
    """
    Cleans LLM-generated SPARQL query by removing markdown backticks.
    """
    return re.sub(r"^```sparql\s*|```$", "", raw_query.strip(), flags=re.MULTILINE).strip()

In [ ]:
sparql_query = generate_sparql_with_mistral(company_name, g.serialize(format="turtle"))
print(sparql_query)
sparql_query = clean_sparql_output(sparql_query)
print(sparql_query)


PREFIX financial: <http://art.uniroma2.it/ontologies/financial-ontology#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX grddl: <http://www.w3.org/2003/g/data-view#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT * WHERE {
  VALUES ?statementType {
    financial:MarketCapitalization
    financial:Dividends
    financial:RevenueVariation
    financial:CostVariation
    financial:IsInCountry
    financial:IsInIndustry
    financial:IsInSector
    financial:IncomeStatement
    financial:BalanceSheet
    financial:CashFlow
    financial:SharesOutstanding
  }
  ?company rdf:label ?label.
  FILTER(regex(str(?label), "Oracle", "i"))
  ?company ?statementType ?statement .
  ?statement ?property ?value .
  FILTER (isLiteral(?value))
}
ORDER BY ?statementType ?statement ?property
PREFIX financial: <http://art.uniroma2.it/ontolo

In [ ]:
financial = Namespace("http://art.uniroma2.it/ontologies/financial-ontology#")
print(financial)

http://art.uniroma2.it/ontologies/financial-ontology#


In [ ]:
import re
from rdflib import Graph, Namespace, BNode
def extract_triples_from_query(query_str: str, ns: Namespace, expected_prefix: str = "financial") -> Graph:
      extract = Graph()
      bnodes = []

    # Find VALUES clause with predicates
      pattern = re.compile(r"VALUES\s+\?\w+\s*{([^}]*)}", re.MULTILINE)
      values_match = pattern.search(query_str)

      if not values_match:
        print("No VALUES block with prefixed predicates found.")
        return extract

      values_block = values_match.group(1)

      # Match entries like financial:Dividends
      predicate_matches = re.findall(r"([a-zA-Z0-9_]+):(\w+)", values_block)

      for prefix, pred in predicate_matches:
        if prefix != expected_prefix:
            print(f"Skipping prefix: {prefix}")
            continue

        # Create synthetic triple: _:s financial:pred _:o
        s = BNode()
        o = BNode()
        p = ns[pred]
        extract.add((s, p, o))

      print(f"Extracted {len(extract)} triples from VALUES block.")
      return extract
print(extract_triples_from_query(sparql_query, financial))

Extracted 11 triples from VALUES block.
[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


In [ ]:
query_graph = extract_triples_from_query(sparql_query, financial)
print(query_graph)

Extracted 11 triples from VALUES block.
[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


In [ ]:
domain_violations = check_domain_violation(query_graph, g)
range_violations = check_range_violation(query_graph, g)

print(domain_violations)
print(range_violations)

[{'type': 'domain', 'property': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#CostVariation'), 'expected_domain': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Company'), 'subject': rdflib.term.BNode('Nfc58ec92fb6a4307b54d4525a6a2c959')}, {'type': 'domain', 'property': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#IsInSector'), 'expected_domain': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Company'), 'subject': rdflib.term.BNode('Nf4399f274dbd4cf0b6694f2a9f1356c9')}, {'type': 'domain', 'property': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#MarketCapitalization'), 'expected_domain': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Company'), 'subject': rdflib.term.BNode('N01116b3fed3541548855f9956404cb62')}, {'type': 'domain', 'property': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#IsInCountry'), 

In [ ]:
complete_violations = domain_violations + range_violations
print(complete_violations)

[{'type': 'domain', 'property': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#CostVariation'), 'expected_domain': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Company'), 'subject': rdflib.term.BNode('Nfc58ec92fb6a4307b54d4525a6a2c959')}, {'type': 'domain', 'property': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#IsInSector'), 'expected_domain': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Company'), 'subject': rdflib.term.BNode('Nf4399f274dbd4cf0b6694f2a9f1356c9')}, {'type': 'domain', 'property': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#MarketCapitalization'), 'expected_domain': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Company'), 'subject': rdflib.term.BNode('N01116b3fed3541548855f9956404cb62')}, {'type': 'domain', 'property': rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#IsInCountry'), 

In [ ]:
explanation = ""  # Initialize explanation as an empty string
for violation in complete_violations:
    explanation += explain_violation(violation) + "\n"  # Concatenate individual explanations

print(explanation)

The property http://art.uniroma2.it/ontologies/financial-ontology#CostVariation has domain http://art.uniroma2.it/ontologies/financial-ontology#Company, but its subject Nfc58ec92fb6a4307b54d4525a6a2c959 does not match or inherit from it.
The property http://art.uniroma2.it/ontologies/financial-ontology#IsInSector has domain http://art.uniroma2.it/ontologies/financial-ontology#Company, but its subject Nf4399f274dbd4cf0b6694f2a9f1356c9 does not match or inherit from it.
The property http://art.uniroma2.it/ontologies/financial-ontology#MarketCapitalization has domain http://art.uniroma2.it/ontologies/financial-ontology#Company, but its subject N01116b3fed3541548855f9956404cb62 does not match or inherit from it.
The property http://art.uniroma2.it/ontologies/financial-ontology#IsInCountry has domain http://art.uniroma2.it/ontologies/financial-ontology#Company, but its subject Nb8d81c90e53940dabb4ba9f8eb18f75a does not match or inherit from it.
The property http://art.uniroma2.it/ontologies

In [ ]:
if explanation == "":
  repaired_query = sparql_query
else:
    repaired_query = repair_query_with_mistral(sparql_query, g.serialize(format="turtle"), explanation, BEE_KEY)
    print(repaired_query)

```sparql
PREFIX financial: <http://art.uniroma2.it/ontologies/financial-ontology#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX grddl: <http://www.w3.org/2003/g/data-view#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT * WHERE {
  VALUES ?statementType {
    financial:MarketCapitalization
    financial:Dividends
    financial:RevenueVariation
    financial:CostVariation
    financial:IsInCountry
    financial:IsInIndustry
    financial:IsInSector
    financial:IncomeStatement
    financial:BalanceSheet
    financial:CashFlow
    financial:SharesOutstanding
  }
  ?company rdf:label ?label.
  FILTER(regex(str(?label), "Oracle", "i"))
  ?company ?statementType ?statement .
  ?statement ?property ?value .
  FILTER (isLiteral(?value))
}
ORDER BY ?statementType ?statement ?property
```


In [ ]:
def clean_sparql_output(raw_query: str) -> str:
    """
    Cleans LLM-generated SPARQL query by removing markdown backticks.
    """
    return re.sub(r"^```sparql\s*|```$|The provided SPARQL query has several issues related to incorrect or mismatched prefixes and properties. Below is the corrected SPARQL query\\:|The main corrections include ensuring that the prefixes are correctly defined and used throughout the query\\. The original query had issues with incorrect or mismatched prefixes\\, which have been fixed\\.", "", raw_query.strip(), flags=re.MULTILINE).strip()

In [ ]:
def extract_sparql_query_only(text: str) -> str:
    """
    Extracts only the SPARQL query from an LLM-generated response.
    Starts from first PREFIX line and stops when all opened braces are closed.
    """
    lines = text.strip().splitlines()
    query_lines = []
    brace_count = 0
    in_query = False

    for line in lines:
        stripped = line.strip()

        # Start collecting at first PREFIX
        if stripped.startswith("PREFIX"):
            in_query = True

        if in_query:
            query_lines.append(line)

            # Track braces to know when WHERE clause is done
            brace_count += stripped.count("{")
            brace_count -= stripped.count("}")

            # Exit when all opened braces are closed and query has started
            if brace_count <= 0 and stripped.startswith("}"):
                break

    return "\n".join(query_lines).strip()



In [ ]:
repaired_query = extract_sparql_query_only(repaired_query)

In [ ]:
print(repaired_query)

PREFIX financial: <http://art.uniroma2.it/ontologies/financial-ontology#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX grddl: <http://www.w3.org/2003/g/data-view#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT * WHERE {
  VALUES ?statementType {
    financial:MarketCapitalization
    financial:Dividends
    financial:RevenueVariation
    financial:CostVariation
    financial:IsInCountry
    financial:IsInIndustry
    financial:IsInSector
    financial:IncomeStatement
    financial:BalanceSheet
    financial:CashFlow
    financial:SharesOutstanding
  }
  ?company rdf:label ?label.
  FILTER(regex(str(?label), "Oracle", "i"))
  ?company ?statementType ?statement .
  ?statement ?property ?value .
  FILTER (isLiteral(?value))
}


In [ ]:
results = g.query(repaired_query)
for row in results:
    print(row)

(rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#CountryGDP'), rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#IsInCountry'), rdflib.term.Literal('25439700.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), rdflib.term.Literal('Oracle', lang='en'), rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#United_States'), rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#Oracle_Corporation'))
(rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#EquityRiskPremium'), rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#IsInCountry'), rdflib.term.Literal('0.05333417', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), rdflib.term.Literal('Oracle', lang='en'), rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-ontology#United_States'), rdflib.term.URIRef('http://art.uniroma2.it/ontologies/financial-onto

In [ ]:
def format_results_for_llm(results) -> str:
    formatted = []
    for row in results:
        row_str = []
        for i, col in enumerate(row):
            # Convert each RDF term into a string
            if hasattr(col, "n3"):
                row_str.append(f"{col.n3()}")
            else:
                row_str.append(str(col))
        formatted.append(" | ".join(row_str))
    return "\n".join(formatted)
formatted_results = format_results_for_llm(results)
print(formatted_results)

<http://art.uniroma2.it/ontologies/financial-ontology#CountryGDP> | <http://art.uniroma2.it/ontologies/financial-ontology#IsInCountry> | "25439700.0"^^<http://www.w3.org/2001/XMLSchema#double> | "Oracle"@en | <http://art.uniroma2.it/ontologies/financial-ontology#United_States> | <http://art.uniroma2.it/ontologies/financial-ontology#Oracle_Corporation>
<http://art.uniroma2.it/ontologies/financial-ontology#EquityRiskPremium> | <http://art.uniroma2.it/ontologies/financial-ontology#IsInCountry> | "0.05333417"^^<http://www.w3.org/2001/XMLSchema#double> | "Oracle"@en | <http://art.uniroma2.it/ontologies/financial-ontology#United_States> | <http://art.uniroma2.it/ontologies/financial-ontology#Oracle_Corporation>
<http://art.uniroma2.it/ontologies/financial-ontology#Inflation> | <http://art.uniroma2.it/ontologies/financial-ontology#IsInCountry> | "2.9"^^<http://www.w3.org/2001/XMLSchema#double> | "Oracle"@en | <http://art.uniroma2.it/ontologies/financial-ontology#United_States> | <http://art.u

# RAG

Now we apply RAG to the model, so that it can provide an answer based on the context provided with the Knowledge Graph and the RDF triples retrievd via SPARQL

In [ ]:
llm = ChatMistralAI(model="mistral-small-latest", api_key=BEE_KEY)

# Prompt template--NEEDS WORK
prompt_template = PromptTemplate(
    input_variables=["results"],
    template=(
        "You are an expert in finance and ontologies. "
        "Given the following SPARQL query result, describe the company's"
        "financial situation, making explicit reference to its Financial Statement\n"
        "Return: \n"
        "- its general situation taking into account its variation in costs and revenues,"
        "  its market capitalisation, its future investments and amortisations, and the amount of dividends it distributes to shareholders\n"
        "- describe the industry and country they are operating in, listing their properties and indexes\n"
        "- its IncomeStatement\n"
        "- its BalanceSheet\n"
        "- its CashFlow\n"
        "- its SharesOutstanding\n"

        "Avoid technical RDF language. Be as specific as possible. For each property or item in the financial statements, report its **value**.\n\n"
        "sparql_graph:\n{results}"
    )
)
chain = LLMChain(llm=llm, prompt=prompt_template)
description_sparql = chain.run(results=formatted_results).strip()

In [ ]:
print(description_sparql)

### Oracle Corporation's Financial Situation

#### General Situation
Oracle Corporation is a technology company operating in the United States. The company's financial situation can be assessed by examining its variation in costs and revenues, market capitalization, future investments, amortizations, and dividends distributed to shareholders.

- **Market Capitalization**: $49,157,040,000
- **Future Investments and Amortizations**: The company has significant capital expenditures amounting to -$6,866,000,000, indicating substantial investments in future growth.
- **Dividends Distributed to Shareholders**: This information is not explicitly provided in the data, but the company's financial health suggests it has the capacity to distribute dividends.

#### Industry and Country
Oracle Corporation operates in the **Software - Infrastructure** industry within the broader **Technology** sector. The company is based in the United States, which has the following economic and demographic propert

In [ ]:
from sentence_transformers import SentenceTransformer, util
from langchain.embeddings import SentenceTransformerEmbeddings
#Chunking the subGraph Text
loader = TextLoader(txt_file)
documents = loader.load()
text_splitter = SemanticChunker(SentenceTransformerEmbeddings(), breakpoint_threshold_type='percentile', breakpoint_threshold_amount=90) # chose which embeddings and breakpoint type and threshold to use
texts = [doc.page_content for doc in documents]
documentation = text_splitter.create_documents(texts)
#Chunking the query result
description_chunks = text_splitter.create_documents([description_sparql])
# Define the embedding model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = SentenceTransformerEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# Create the vector store
all_documents = documentation + description_chunks
vector = FAISS.from_documents(all_documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()
# Define LLM
model = ChatMistralAI(model = "mistral-large-latest",mistral_api_key=BEE_KEY)
prompt = ChatPromptTemplate.from_template("""You are Hiroshi Abe, a helpful and expert AI Financial Assistant specializing in fundamental analysis of companies.

Your role is to analyze a company’s financial health based on the provided data sources. You MUST only use the two sources below to generate your answer.

When analyzing a company's financial situation, follow these steps:

1. **PEST Analysis**
   Perform a Political, Economic, Social, and Technological (PEST) analysis relevant to the firm.

2. **Financial Ratios Evaluation**
   Calculate and interpret:
   - **Return on Equity (ROE)**
   - **Return on Investment (ROI)**
   - **Return on Assets (ROA)**
   Use the following formulas for assessment:
   • ROE ≥ *r_f* + *β_i* × (*r_M* − *r_f*)
   • ROI ≥ ROE
   Where:
   - *r_f* = 4% (US Treasury Bond Return Rate)
   - *r_M* = 23.63% (Technology Sector Market Return Rate)
   - *β_i* = Industry Beta
   Explain each computation you do, highlighting every line item used
   Conclude whether the financial situation is **Good** or **Not Good** based on these thresholds.

3. **Forecasting**
   Provide 5-year forecasted:
   - **Balance Sheets**
   - **Income Statements**
   These forecasts must be based on estimated changes in **costs**, **revenues**, and **amortization schedules**.

4. **Firm Valuation (DCF Method)**
   Compute the firm’s valuation using the **Discounted Cash Flow (DCF)** method.

---

 **Answer the question based only on the provided context:**
  {context}

---

 **User Question:**
{input}"""
)

<ipython-input-43-1a9ae86994bc>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  text_splitter = SemanticChunker(SentenceTransformerEmbeddings(), breakpoint_threshold_type='percentile', breakpoint_threshold_amount=90) # chose which embeddings and breakpoint type and threshold to use
<ipython-input-43-1a9ae86994bc>:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(SentenceTransformerEmbeddings(), breakpoint_threshold_type='percentile', breakpoin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": question})
print(response["answer"])

To analyze Oracle Corporation's financial situation, we will follow the structured approach involving PEST Analysis, Financial Ratios Evaluation, Forecasting, and Firm Valuation using the Discounted Cash Flow (DCF) method.

### 1. PEST Analysis

**Political:**
- The United States has a stable political environment, which is favorable for businesses.
- Regulations and policies in the technology sector are generally supportive of innovation and growth.

**Economic:**
- The US GDP is $25,439,700,000,000, indicating a strong economy.
- The equity risk premium is 5.33%, reflecting the expected return over the risk-free rate.
- Inflation is at 2.9%, which is moderate and manageable.
- Moody's Rating: Aa3 and S&P Rating: AA+ indicate a strong creditworthiness.

**Social:**
- The median age is 38.9 years, with 17.43% of the population over 65, suggesting a mature workforce.
- The GDI (Gender Development Index) is 78.8, indicating a moderately equal society.
- The fertility rate is 1.6, which i

As "GSM-Symbolic" by Mirzadeh reports, LLMs do not understand how math works. Therefore, it was decided to use another prompt for context analysis and Balance Sheet Retrieval.

In [ ]:
from sentence_transformers import SentenceTransformer, util
from langchain.embeddings import SentenceTransformerEmbeddings
#Chunking the subGraph Text
loader = TextLoader(txt_file)
documents = loader.load()
text_splitter = SemanticChunker(SentenceTransformerEmbeddings(), breakpoint_threshold_type='percentile', breakpoint_threshold_amount=90) # chose which embeddings and breakpoint type and threshold to use
texts = [doc.page_content for doc in documents]
documentation = text_splitter.create_documents(texts)
#Chunking the query result
description_chunks = text_splitter.create_documents([description_sparql])
# Define the embedding model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = SentenceTransformerEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# Create the vector store
all_documents = documentation + description_chunks
vector = FAISS.from_documents(all_documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()
# Define LLM
model = ChatMistralAI(model = "mistral-large-latest",mistral_api_key=BEE_KEY)
prompt = ChatPromptTemplate.from_template("""You are Hiroshi Abe, a helpful and expert AI Financial Assistant specializing in fundamental analysis of companies.

Your role is to provide context information and help analysts make fundamental analysis. You MUST only use the two sources below to generate your answer.

When retrieving data about a company, follow these steps:

1. **PEST Analysis**
   Analyse the company from a macro perspective, according to the PEST Analysis paradigm:
   - **Political**
   - **Economic**
   - **Social**
   - **Technological**
  For each point, write a brief comment on the company's situation.

2. **General Data**
Return general information, including:
- **Market Capitalization**
- **Industry**
- **Country**
- **Variation in Costs and Revenues**
- **Future Investments**
- **Shares Outstanding**.

3. **Financial Statements**
Return its Financial Statement:
   - **Income Statement**
   - **Balance Sheet**
   - **Cash Flow**

Use bullet points or structured paragraphs for each section. Keep the language clear and professional. Be concise but thorough.
---

 **Answer the question based only on the provided context:**
  {context}

---

 **User Question:**
{input}"""
)

<ipython-input-45-d9a0dde33cf6>:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(SentenceTransformerEmbeddings(), breakpoint_threshold_type='percentile', breakpoint_threshold_amount=90) # chose which embeddings and breakpoint type and threshold to use


In [ ]:
# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": question})
print(response["answer"])

### Oracle Corporation's Financial Situation

#### PEST Analysis

**Political:**
- Oracle operates in the United States, which has a stable political environment. The country's Moody's Rating is Aa3 and S&P Rating is AA+, indicating a strong and reliable political and economic framework.

**Economic:**
- The U.S. has a GDP of $25,439,700,000,000 and an equity risk premium of 5.33%, reflecting a robust economic environment. The inflation rate is 2.9%, and the country's research expenditure as a percentage of GDP is 3.46%, suggesting a focus on innovation and growth.

**Social:**
- The U.S. has a median age of 38.9 years and a population over 65 of 17.43%. The GDI is 78.8, and the fertility rate is 1.6. These demographic factors influence consumer behavior and labor market dynamics, affecting Oracle's market strategies and workforce planning.

**Technological:**
- Oracle's significant investment in research and development, as indicated by the country's focus on research, positions it we